# Load data extraction


Data has been extracted from the selected studies. 
This file permits to load the extracted data, post-process it, and create LaTeX code from it.

This notebook is organized as follows:
1. In Section 1, we create tables and timelines for studies in the group YY
2. In Section 2, we create tables and timelines for studies in the group NY
3. In Section 3, we create tables and timelines for studies in the group YN
4. In Section 4, we create lists of links
5. In Section 5, we create tables for methods and tools not selected in the review process.

***

**Importing libraries:**

In [1]:
import pandas as pd
import os
from pprint import pprint
from collections import Counter
import collections

**Usefull funtions and variables:**

In [2]:
TABLE_PARAM=''

def transpose_table(df):
    df = pd.DataFrame(df.values.T[1:], columns=df.id.tolist())
    return(df)

links_list = []

# for LaTeX acronyms:
abbrev_list_lower = ['cnn', 'epm', 'flop', 'ict', 'mac', 'nas', 'nlp', 'nn', 'nu', 'nvml', 'pmc', 'ptx', 'rapl', 'slr', 'smc', 'tdp']
abbrev_list_upper = [w.upper() for w in abbrev_list_lower]

## 1. YY studies

### 1.1 Making the tables

In [3]:
_path = os.path.join('data_YY.xlsx')
df_YY = pd.DataFrame(pd.read_excel(_path))

In [4]:
# transpose the dataframe:
df_YY = transpose_table(df_YY)

df_YY = df_YY.sort_values('year', ascending=False)
df_YY['name'] = df_YY['name'].replace(['none'], 'NU')
df_YY['constraints'] = df_YY['constraints'].replace(['none'], 'no')
df_YY['available'] = df_YY['available'].replace(['none'], 'no')
df_YY['general'] = df_YY['fulltext_id'].apply(lambda x: '\cite{' + str(x) + '}').astype(str) + ', ' + df_YY['year'].astype(str) + ', ' + df_YY['name'].astype(str)

df_YY.loc[df_YY['available_link'].str.contains('http'), 'available'] = df_YY['available'].astype(str) + ' \\href{' + df_YY['available_link'].astype(str) + '}{\\ref*{link-' + df_YY['fulltext_id'].astype(str) + '}}' 
df_YY.loc[df_YY['available_link_bis'].str.contains('http'), 'available'] =  df_YY['available'].astype(str) + ' \\href{' + df_YY['available_link_bis'].astype(str) + '}{\\ref*{link-' + df_YY['fulltext_id'].astype(str) + '-bis}}'

df_YY.loc[df_YY['available_link'].str.contains('http'), 'for_links_list'] = '\\item \\label{link-' + df_YY['fulltext_id'].astype(str) + '} \\url{' + df_YY['available_link'].astype(str) + '}'
df_YY.loc[df_YY['available_link_bis'].str.contains('http'), 'for_links_list_bis'] = '\\item \\label{link-' + df_YY['fulltext_id'].astype(str) + '-bis} \\url{' + df_YY['available_link_bis'].astype(str) + '}'

df_YY['for_timeline'] = df_YY['name'].astype(str) + ' ' + df_YY['fulltext_id'].apply(lambda x: '\cite{' + str(x) + '}').astype(str)

df_YY['detail_1'] = df_YY['detail_1'].astype(str) + ', accounting for ' + df_YY['detail_2'].astype(str)

df_YY.loc[df_YY['note'].notna(), 'detail_1'] = df_YY['note'].astype(str) + ' ' + df_YY['detail_1'].astype(str)
df_YY.loc[df_YY['note_2'].notna(), 'general'] = df_YY['general'].astype(str) + ', ' + df_YY['note_2'].astype(str)

columns_new_names = {'general': 'study', 'detail_1': 'detail', 'citations':'cites', 'constraints':'constraints', 'task':'target task', 'available':'available'}

for key in columns_new_names:
    columns_new_names[key] = '\\bfseries ' + columns_new_names[key]

def create_latex_YY(df_YY, category, disp):
    list_cols = ['general', 'detail_1', 'task', 'constraints', 'available', 'citations']
    a = '>{\\raggedright\\arraybackslash}'
    latex_col_width = '|'+a+'p{2cm}|p{5.9cm}|'+a+'p{1.85cm}|'+a+'p{1.95cm}|'+a+'p{1.5cm}|p{0.75cm}|'

    df_YY_category = df_YY[df_YY.category == category][list_cols]

    if disp:
        display(df_YY_category)
    df_YY_category = df_YY_category.rename(columns=columns_new_names)
    latex_YY = df_YY_category.to_latex(index=False).replace('llllll', latex_col_width)
    # latex_YY = latex_YY.replace('\\\\\n\\cite', '\\vspace{2mm}\\\\\\\n\\cite')

    if group == 'other':
        caption='YY studies without group'
    else:
        caption="YY studies in the group ``" + category + "''"
    lab='YY-'+'-'.join(category.split())

    # for the version with tables:
    # latex_YY = '\\begin{table}['+ TABLE_PARAM +'] \n\\begin{minipage}{\\textwidth} \n\\centering \n' + latex_YY + '\\caption{'+ caption +'} \n\\label{tab:' + lab + '} \n\\end{minipage} \n\\end{table}'
    
    # for the version with paragraphs:
    latex_YY = '\\paragraph{'+caption+ '} \n \\label{tab:'+lab+'} \n' + latex_YY
    latex_YY = latex_YY.replace('{tabular}', '{longtable}')
    latex_YY = latex_YY.replace('\\midrule', '\\midrule \n\\endhead')

    for w in abbrev_list_upper:
        latex_YY = latex_YY.replace(w, '\\acrshort{'+w.lower()+'}')

    file_name = '-'.join(category.split())
    with open(os.path.join('tables', 'YY-' + file_name + '.tex'), 'w') as f:
        f.write(latex_YY)

    return(latex_YY)


groups = ['analytical estimation model', 'data-based estimation model', 'on-chip sensors',\
        'analytical and data-based estimation model','analytical estimation model and on-chip sensors',\
        'other']
description=[]
for group in groups:
    print(group, ':')
    latex_YY = create_latex_YY(df_YY, group, disp=False)
    print(latex_YY)
    print('\n')

    if group != 'other':
        description.append("\\ref{tab:YY-" + '-'.join(group.split()) + "} for the group `" + group + "'")
    else:
        description.append("\\ref{tab:YY-" + '-'.join(group.split()) + '} for studies without group')
description = ', '.join(description)
description += '.%'
print(description)

file_name = 'YY-description'
with open(os.path.join('tables', file_name + '.tex'), 'w') as f:
    f.write(description)

analytical estimation model :
\paragraph{YY studies in the group ``analytical estimation model''} 
 \label{tab:YY-analytical-estimation-model} 
\begin{longtable}{|>{\raggedright\arraybackslash}p{2cm}|p{5.9cm}|>{\raggedright\arraybackslash}p{1.85cm}|>{\raggedright\arraybackslash}p{1.95cm}|>{\raggedright\arraybackslash}p{1.5cm}|p{0.75cm}|}
\toprule
\bfseries study & \bfseries detail & \bfseries target task & \bfseries constraints & \bfseries available & \bfseries cites \\
\midrule 
\endhead
\cite{desislavov2023}, 2023, \acrshort{nu} & Model with \acrshort{flop} count as input, accounting for computations only, on CPU or GPU & any & no & no & 15 \\
\cite{lemaire2022}, 2022, \acrshort{nu} & Model with \acrshort{nn} architecture as inputs, uses energy consumption of single operations drawn from the literature, and memory size, accounting for CPU or accelerator & Spiking and non-spiking \acrshort{nn} inference & no & code and models upon request & 11 \\
\cite{lannelongue2021}, 2021, GA & (Gr

### 1.2. Timeline

In [5]:
grouped = df_YY.groupby('year')['for_timeline'].apply(list)
dict_timeline = grouped.to_dict()

In [6]:
def post_process_timeline(dict_timeline):
    d = {}
    res = {}
    for key,value in dict_timeline.items():
        d[key]={}
        
        for v in value:
            if v.split()[0] not in d[key]:
                d[key][v.split()[0]] = []
            d[key][v.split()[0]].append(v.split()[1])
        
            
    for key,value in d.items():
        res[key]=[]
        
        #order values by keys
        value = collections.OrderedDict(sorted(value.items()))

        for k,v in value.items():

            res[key].append(k + ' ' + ' '.join(v))
    print(res)

    return(res)

dict_timeline = post_process_timeline(dict_timeline)


{2017: ['DNNEET \\cite{yang2017}', 'NeuralP \\cite{cai2017}'], 2018: ['SyNERGY \\cite{rodrigues2018}'], 2019: ['CC \\cite{lottick2019}', 'MLCI \\cite{lacoste2019}'], 2020: ['CT \\cite{anthony2020}', 'Cumulator \\cite{trebaol2020}', 'EIT \\cite{henderson2020}'], 2021: ['Accelergy \\cite{wang2021}', 'EcoML \\cite{igescu2021}', 'EnergyNN \\cite{goel2021}', 'GA \\cite{lannelongue2021}', 'NU \\cite{metz2021}', 'Pommel \\cite{montgomerie-corcoran2021}'], 2022: ['BT \\cite{carastan-santos2022}', 'Eco2AI \\cite{budennyy2022}', 'NU \\cite{lemaire2022} \\cite{metz2022b} \\cite{metz2022a} \\cite{lahmer2022}'], 2023: ['DLEE \\cite{getzner2023}', 'NU \\cite{desislavov2023} \\cite{qiu2023} \\cite{dariol2023} \\cite{ortega2023}', 'ZeusM \\cite{you2023}']}


In [7]:
dict_year_point = {}
cpt = 0
step = 2
for key in dict_timeline.keys():
    dict_year_point[key] = str(cpt)
    cpt += step

a = ''
for year, studies in dict_timeline.items():
    if year == 2020:
        aa = studies[1]
        ab = studies[0]+', '+ studies[2]
        ac = ab + '\\\\ ' + aa
    elif year == 2021:
        aa = ', '.join(studies[:len(studies)//3])
        ab = ', '.join(studies[len(studies)//3:2*len(studies)//3])
        abb = ', '.join(studies[2*len(studies)//3:])
        ac = aa + '\\\\ ' + ab + '\\\\ ' + abb  
    elif year == 2022:
        aa = studies[0]+', '+ studies[1]
        ab = studies[2]
        ac = aa + '\\\\ ' + ab
    elif year == 2023:
        aa = studies[1]
        ab = studies[0]+', '+ studies[2]
        ac = aa + '\\\\ ' + ab
    elif len(studies)>=2:
        aa = ', '.join(studies[:len(studies)//2])
        ab = ', '.join(studies[len(studies)//2:])
        ac = aa + '\\\\ ' + ab 
    else:
        ac = ', '.join(studies)
    if year == 2021:
        ad = '\\node[above right, align=left, rotate=45] at (8.3,0) {' + ac + '}; \n'
    else:
        ad = '\\node[above right, align=left, rotate=45] at (' + dict_year_point[year] + ',0) {' + ac + '}; \n'
    a += ad

ba = ', '.join([f"{v}/{k}" for k, v in dict_year_point.items()])

ba + '\n'

b = '\\begin{tikzpicture} \n\
\\draw[->, thick] (0,0) -- (15,0); \n\
\\foreach \\x/\\year in {' + ba + '} { \n\
    \\draw[shift={(\\x,0)}, color=black] (0pt,2pt) -- (0pt,-2pt); \n\
    \\node[below] at (\\x, 0) {\\year}; \n\
    } \n'

c = '\\end{tikzpicture}'


d = b + a + c

print(d)

with open(os.path.join('tables', 'YY-timeline.tex'), 'w') as f:
    f.write(d)


\begin{tikzpicture} 
\draw[->, thick] (0,0) -- (15,0); 
\foreach \x/\year in {0/2017, 2/2018, 4/2019, 6/2020, 8/2021, 10/2022, 12/2023} { 
    \draw[shift={(\x,0)}, color=black] (0pt,2pt) -- (0pt,-2pt); 
    \node[below] at (\x, 0) {\year}; 
    } 
\node[above right, align=left, rotate=45] at (0,0) {DNNEET \cite{yang2017}\\ NeuralP \cite{cai2017}}; 
\node[above right, align=left, rotate=45] at (2,0) {SyNERGY \cite{rodrigues2018}}; 
\node[above right, align=left, rotate=45] at (4,0) {CC \cite{lottick2019}\\ MLCI \cite{lacoste2019}}; 
\node[above right, align=left, rotate=45] at (6,0) {CT \cite{anthony2020}, EIT \cite{henderson2020}\\ Cumulator \cite{trebaol2020}}; 
\node[above right, align=left, rotate=45] at (8.3,0) {Accelergy \cite{wang2021}, EcoML \cite{igescu2021}\\ EnergyNN \cite{goel2021}, GA \cite{lannelongue2021}\\ NU \cite{metz2021}, Pommel \cite{montgomerie-corcoran2021}}; 
\node[above right, align=left, rotate=45] at (10,0) {BT \cite{carastan-santos2022}, Eco2AI \cite{budenny

## 2. NY studies

### 2.1. Making the tables

In [8]:
_path = os.path.join('data_NY.xlsx')
df_NY = pd.DataFrame(pd.read_excel(_path))


# transpose the dataframe:
df_NY = transpose_table(df_NY)
df_NY

df_NY = df_NY.sort_values('year', ascending=False)
df_NY['general'] = df_NY['fulltext_id'].apply(lambda x: '\cite{' + str(x) + '}').astype(str) + ', ' + df_NY['year'].astype(str)
df_NY['for_timeline'] = df_NY['fulltext_id'].apply(lambda x: '\cite{' + str(x) + '}').astype(str)

columns_new_names = {'general': 'study', 'detail': 'detail', 'task':'ML task', 'setup':'setup'}
for key in columns_new_names:
    columns_new_names[key] = '\\bfseries ' + columns_new_names[key]

def create_latex_NY(df_NY, category, disp):
    list_cols = ['general', 'detail', 'task', 'setup']
    a = '>{\\raggedright\\arraybackslash}'
    latex_col_width = '|'+a+'p{0.85cm}|'+a+'p{4cm}|'+a+'p{6cm}|'+a+'p{3.75cm}|'

    df_NY_category = df_NY[df_NY.category == category][list_cols]

    if disp:
        display(df_NY_category)
    df_NY_category = df_NY_category.rename(columns=columns_new_names)
    latex_NY = df_NY_category.to_latex(index=False).replace('llll', latex_col_width)

    if group == 'other':
        caption='NY studies without group'
    else:
        caption="NY studies in the group ``" + category + "''"
    lab='NY-'+'-'.join(category.split())

    # for the version with paragraphs:
    latex_NY = '\\paragraph{'+caption+ '} \n \\label{tab:'+lab+'} \n' + latex_NY
    latex_NY= latex_NY.replace('{tabular}', '{longtable}')
    latex_NY = latex_NY.replace('\\midrule', '\\midrule \n\\endhead')

    for w in abbrev_list_upper:
        latex_NY = latex_NY.replace(w, '\\acrshort{'+w.lower()+'}')

    file_name = '-'.join(category.split())
    with open(os.path.join('tables', 'NY-' + file_name + '.tex'), 'w') as f:
        f.write(latex_NY)

    return(latex_NY)

groups = ['measurement', 'analytical estimation model', 'on-chip sensors', 'data-based estimation model',\
          'other']
description=[]
for group in groups:
    print(group, ':')
    latex_NY = create_latex_NY(df_NY, group, disp=False)
    print(latex_NY)
    print('\n')

    if group != 'other':
        description.append("\\ref{tab:NY-" + '-'.join(group.split()) + "} for the group `" + group + "'")
    else:
        description.append("\\ref{tab:NY-" + '-'.join(group.split()) + '} for studies without group')
description = ', '.join(description)
description += '.%'
print(description)

file_name = 'NY-description'
with open(os.path.join('tables', file_name + '.tex'), 'w') as f:
    f.write(description)


measurement :
\paragraph{NY studies in the group ``measurement''} 
 \label{tab:NY-measurement} 
\begin{longtable}{|>{\raggedright\arraybackslash}p{0.85cm}|>{\raggedright\arraybackslash}p{4cm}|>{\raggedright\arraybackslash}p{6cm}|>{\raggedright\arraybackslash}p{3.75cm}|}
\toprule
\bfseries study & \bfseries detail & \bfseries ML task & \bfseries setup \\
\midrule 
\endhead
\cite{hauschild2023}, 2023 & \acrshort{epm} JT-TC66C & inference with \acrshort{cnn}s: MobileNetV2, \acrshort{nas}NetMobile, ResNet (50, 101), VGG (16, 19) & edge, server \\
\cite{trihinas2022}, 2022 & \acrshort{epm} & inference with \acrshort{cnn} for object detection (on ImageNet) & edge  \\
\cite{machado2022}, 2022 & \acrshort{epm}/sensors on the board & inference with YOLOv5 for object classification & edge (Nvidia Jetson Nano board) \\
\cite{hampau2022}, 2022 & \acrshort{epm} Monsoon's High Voltage Power Monitor & inference with Image Classifyer (on MNIST, Emotion, CIFAR10) and YOLO & edge \\
\cite{hesse2021}, 20

### 2.2. Timeline

In [9]:
grouped = df_NY.groupby('year')['for_timeline'].apply(list)
dict_timeline = grouped.to_dict()

dict_year_point = {}
cpt = 0
step = 2
for key in dict_timeline.keys():
    dict_year_point[key] = str(cpt)
    cpt += step

a = ''
for year, studies in dict_timeline.items():
    if year == 2021 or year == 2023:
        aa = ', '.join(studies[:len(studies)//3])
        ab = ', '.join(studies[len(studies)//3:2*len(studies)//3])
        abb = ', '.join(studies[2*len(studies)//3:])
        ac = aa + '\\\\ ' + ab + '\\\\ ' + abb  
    elif len(studies)>2:
        aa = ', '.join(studies[:len(studies)//2])
        ab = ', '.join(studies[len(studies)//2:])
        ac = aa + '\\\\ ' + ab 
    else:
        ac = ', '.join(studies)
    if year == 2021:
        ad = '\\node[above right, align=left, rotate=45] at (8.3,0) {' + ac + '}; \n'
    elif year == 2023:
        ad = '\\node[above right, align=left, rotate=45] at (12.3,0) {' + ac + '}; \n'
    else:
        ad = '\\node[above right, align=left, rotate=45] at (' + dict_year_point[year] + ',0) {' + ac + '}; \n'
    a += ad

ba = ', '.join([f"{v}/{k}" for k, v in dict_year_point.items()])

ba + '\n'

b = '\\begin{tikzpicture} \n\
\\draw[->, thick] (0,0) -- (15,0); \n\
\\foreach \\x/\\year in {' + ba + '} { \n\
    \\draw[shift={(\\x,0)}, color=black] (0pt,2pt) -- (0pt,-2pt); \n\
    \\node[below] at (\\x, 0) {\\year}; \n\
    } \n'

c = '\\end{tikzpicture}'


d = b + a + c

print(d)

with open(os.path.join('tables', 'NY-timeline.tex'), 'w') as f:
    f.write(d)

\begin{tikzpicture} 
\draw[->, thick] (0,0) -- (15,0); 
\foreach \x/\year in {0/2016, 2/2017, 4/2019, 6/2020, 8/2021, 10/2022, 12/2023} { 
    \draw[shift={(\x,0)}, color=black] (0pt,2pt) -- (0pt,-2pt); 
    \node[below] at (\x, 0) {\year}; 
    } 
\node[above right, align=left, rotate=45] at (0,0) {\cite{li2016a}}; 
\node[above right, align=left, rotate=45] at (2,0) {\cite{rungsuptaweekoon2017}}; 
\node[above right, align=left, rotate=45] at (4,0) {\cite{wang2019}\\ \cite{strubell2019}, \cite{mcintosh2019}}; 
\node[above right, align=left, rotate=45] at (6,0) {\cite{rodrigues2020}, \cite{jurj2020}\\ \cite{holly2020}, \cite{yao2021}}; 
\node[above right, align=left, rotate=45] at (8.3,0) {\cite{arnautovic2021}, \cite{guo2021}\\ \cite{naidu2021}, \cite{desislavov2021}\\ \cite{sun2021}, \cite{hesse2021}, \cite{canilang2021}}; 
\node[above right, align=left, rotate=45] at (10,0) {\cite{trihinas2022}\\ \cite{machado2022}, \cite{hampau2022}}; 
\node[above right, align=left, rotate=45] at (1

## 3. YN studies

### 3.1. Making the tables

In [10]:
_path = os.path.join('data_YN.xlsx')
df_YN = pd.DataFrame(pd.read_excel(_path))


# transpose the dataframe:
df_YN = transpose_table(df_YN)

df_YN = df_YN.sort_values('year', ascending=False)

df_YN['name'] = df_YN['name'].replace(['none'], 'NU')

df_YN['general'] = df_YN['fulltext_id'].apply(lambda x: '\cite{' + str(x) + '}').astype(str) + ', ' + df_YN['year'].astype(str) + ', ' + df_YN['name'].astype(str)
df_YN['for_timeline'] = df_YN['name'].astype(str) + ' ' + df_YN['fulltext_id'].apply(lambda x: '\cite{' + str(x) + '}').astype(str)

#  If want to add the note:
df_YN.loc[df_YN['note_2'].notna(), 'detail'] = ' (' + df_YN['note_2'].astype(str) + ') ' + df_YN['detail'].astype(str)
df_YN.loc[df_YN['note_1'].notna(), 'detail'] = df_YN['note_1'].astype(str) + ' -- ' + df_YN['detail'].astype(str)

df_YN.loc[df_YN['available_link'].str.contains('http'), 'available'] = df_YN['available'].astype(str) + ' \\href{' + df_YN['available_link'].astype(str) + '}{\\ref*{link-' + df_YN['fulltext_id'].astype(str) + '}}'
df_YN.loc[df_YN['available_link'].str.contains('http'), 'for_links_list'] = '\\item \\label{link-' + df_YN['fulltext_id'].astype(str) + '} \\url{' + df_YN['available_link'].astype(str) + '}'

df_YN.loc[df_YN['available'].ne('none'), 'detail'] = df_YN['detail'].astype(str) + '; \\textbf{available:} ' + df_YN['available'].astype(str)

columns_new_names = {'general': 'study', 'detail': 'detail', 'available':'available', 'citations':'cites'}
for key in columns_new_names:
    columns_new_names[key] = '\\bfseries ' + columns_new_names[key]

def create_latex_YN(df_YN, category, disp, date=None, above=True):
    list_cols = ['general', 'detail', 'citations']
    a = '>{\\raggedright\\arraybackslash}'
    latex_col_width = '|'+a+'p{2.75cm}|p{11.75cm}|p{0.7cm}|'

    df_YN_category = df_YN[df_YN.category == category]

    df_YN_category = df_YN_category[list_cols]

    print('nb: ', df_YN_category.shape[0])

    if disp:
        display(df_YN_category)
    df_YN_category = df_YN_category.rename(columns=columns_new_names)
    latex_YN = df_YN_category.to_latex(index=False).replace('lll', latex_col_width)

    if group == 'other':
        caption='YN studies without group'
    elif group == 'hybrid':
        caption='YN studies in a mixed group'
    else:
        caption="YN studies in the group ``" + category + "''"
    lab='YN-'+'-'.join(category.split())

    # for the version with paragraphs:
    latex_YN = '\\paragraph{'+caption+ '} \n \\label{tab:'+lab+'} \n' + latex_YN
    latex_YN = latex_YN.replace('{tabular}', '{longtable}')
    latex_YN = latex_YN.replace('\\midrule', '\\midrule \n\\endhead')

    file_name = '-'.join(category.split())
    if date:
        if above:
            file_name += '-' + str(date) + 'above'
        else:
            file_name += '-' + str(date) + 'below'

    for w in abbrev_list_upper:
        latex_YN = latex_YN.replace(w, '\\acrshort{'+w.lower()+'}')
    
    with open(os.path.join('tables', 'YN-' + file_name + '.tex'), 'w') as f:
        f.write(latex_YN)

    return(latex_YN)


groups = ['measurement', 'analytical estimation model', 'data-based estimation model',\
        'on-chip sensors', 'hybrid', 'other']
description=[]
above=False
for group in groups:
    print(group, ':')
    latex_YN = create_latex_YN(df_YN, group, disp=False)
    print(latex_YN)
    print('\n')

    print(above)

    if group == 'other':
        description.append("\\ref{tab:YN-" + '-'.join(group.split()) + '} for studies without group')
    elif group == 'hybrid':
        description.append("\\ref{tab:YN-" + '-'.join(group.split()) + '} for studies in a mixed group')
    else:
        description.append("\\ref{tab:YN-" + '-'.join(group.split()) + "} for the group `" + group + "'")
description = ', '.join(description)
description += '.%'
print(description)

file_name = 'YN-description'
with open(os.path.join('tables', file_name + '.tex'), 'w') as f:
    f.write(description)

measurement :
nb:  6
\paragraph{YN studies in the group ``measurement''} 
 \label{tab:YN-measurement} 
\begin{longtable}{|>{\raggedright\arraybackslash}p{2.75cm}|p{11.75cm}|p{0.7cm}|}
\toprule
\bfseries study & \bfseries detail & \bfseries cites \\
\midrule 
\endhead
\cite{hankel2016}, 2016, \acrshort{nu} & sensors into motherboard power grid to measure power draw of components, for CPU, RAM, Network, Disk, Power Supply and System & 0 \\
\cite{enam2013}, 2013, \acrshort{nu} & \acrshort{epm} based on the Arduino board, for the whole system & 0 \\
\cite{ferreira2013}, 2013, SEFLab &  (Software Energy Footprint Lab) based on separate measurements of CPU, RAM, Fans, Disk and Motherboard, for execution of a software; \textbf{available:} code \href{https://github.com/SEFLab}{\ref*{link-ferreira2013}} & 64 \\
\cite{piga2011}, 2011, \acrshort{nu} & custom made board measures the computer power via current transducers, a data acquisition device, and a software that controls the framework, for t

### 3.2. Timeline

In [11]:
grouped = df_YN[df_YN['year'] <= 2015].groupby('year')['for_timeline'].apply(list)
dict_timeline = grouped.to_dict()

In [12]:
def post_process_timeline(dict_timeline):
    d = {}
    res = {}
    for key,value in dict_timeline.items():
        d[key]={}
        
        for v in value:
            if v.split()[0] not in d[key]:
                d[key][v.split()[0]] = []
            d[key][v.split()[0]].append(v.split()[1])
        
            
    for key,value in d.items():
        res[key]=[]
        
        for k,v in value.items():

            res[key].append(k + ' ' + ' '.join(v))
    print(res)

    return(res)

dict_timeline = post_process_timeline(dict_timeline)

{2002: ['SoftWatt \\cite{gurumurthi2002}'], 2008: ['NU \\cite{lewis2008}'], 2009: ['NU \\cite{singh2009} \\cite{ma2009} \\cite{spellmann2009}'], 2010: ['LikwidPM \\cite{treibig2010}', 'NU \\cite{zamani2010}'], 2011: ['NU \\cite{suda2011} \\cite{basmadjian2011} \\cite{piga2011} \\cite{matsumoto2011} \\cite{chen2011}'], 2012: ['ECAT \\cite{chen2012}', 'eprof \\cite{schubert2012}'], 2013: ['NU \\cite{enam2013} \\cite{peng2013} \\cite{singh2013}', 'PowerAPI \\cite{bourdon2013}', 'SEFLab \\cite{ferreira2013}'], 2014: ['Jalen \\cite{noureddine2014a}', 'BitWatts \\cite{colmant2014}', 'NU \\cite{storlie2014} \\cite{kim2014}', 'JalenUnit \\cite{noureddine2014}'], 2015: ['NU \\cite{gutierrez2015a} \\cite{foo2015} \\cite{harton2015} \\cite{gutierrez2015b}', 'E-Surgeon \\cite{noureddine2015}']}


In [13]:
dict_year_point = {}
cpt = -1
step = 1.75
for key in dict_timeline.keys():
    dict_year_point[key] = str(cpt)
    cpt += step

a = ''
for year, studies in dict_timeline.items():
    if len(studies)>=2:
        aa = ', '.join(studies[:len(studies)//2])
        ab = ', '.join(studies[len(studies)//2:])
        ac = aa + '\\\\ ' + ab 
    else:
        ac = ', '.join(studies)

    # ac = ', '.join(studies)

    ad = '\\node[above right, align=left, rotate=45] at (' + dict_year_point[year] + ',0) {' + ac + '}; \n'
    a += ad

ba = ', '.join([f"{v}/{k}" for k, v in dict_year_point.items()])

ba + '\n'

b = '\\begin{tikzpicture} \n\
\\draw[dashed, thick] (-1.5,0) -- (1,0); \n\
\\draw[thick] (1,0) -- (13.5,0); \n\
\\draw[dashed, thick] (13.5,0) -- (14,0); \n\
\\foreach \\x/\\year in {' + ba + '} { \n\
    \\draw[shift={(\\x,0)}, color=black] (0pt,2pt) -- (0pt,-2pt); \n\
    \\node[below] at (\\x, 0) {\\year}; \n\
    } \n'


d = b + a

d+= '\n\n'

In [14]:
grouped = df_YN[df_YN['year'] > 2015].groupby('year')['for_timeline'].apply(list)
dict_timeline = grouped.to_dict()
dict_timeline['2019'] = []

dict_timeline = {k: dict_timeline[k] for k in sorted(dict_timeline, key=int)}

In [15]:
def post_process_timeline(dict_timeline):
    d = {}
    res = {}
    for key,value in dict_timeline.items():
        d[key]={}
        
        for v in value:
            if v.split()[0] not in d[key]:
                d[key][v.split()[0]] = []
            d[key][v.split()[0]].append(v.split()[1])
        
            
    for key,value in d.items():
        res[key]=[]
        
        for k,v in value.items():

            res[key].append(k + ' ' + ' '.join(v))
    print(res)

    return(res)

dict_timeline = post_process_timeline(dict_timeline)

{2016: ['TEEC \\cite{acar2016c} \\cite{acar2016b} \\cite{acar2016a}', 'NU \\cite{li2016b} \\cite{hankel2016} \\cite{veni2016} \\cite{park2016}'], 2017: ['NU \\cite{liu2017} \\cite{alzamil2017} \\cite{jiang2017}', 'Powerstat \\cite{becker2017}'], 2018: ['TVAKSHAS \\cite{naren2018}', 'NU \\cite{fu2018} \\cite{dutta2018}'], '2019': [], 2020: ['NU \\cite{lin2020} \\cite{kloh2020} \\cite{karantoumanis2020}'], 2021: ['NU \\cite{shahid2021b} \\cite{shahid2021a} \\cite{aboubakar2021}', 'Phantom \\cite{montanana-aliaga2021}'], 2022: ['PMT \\cite{corda2022}', 'MuMMI \\cite{wu2022}', 'DeepPM \\cite{shim2022}', 'NU \\cite{morlans2022}', 'PJ,JJX \\cite{noureddine2022}'], 2023: ['NU \\cite{alavani2023}', 'ESAVE \\cite{pathania2023}']}


In [16]:
dict_year_point = {}
cpt = 0
step = 1.75
for key in dict_timeline.keys():
    dict_year_point[key] = str(cpt)
    cpt += step

delta = str(-4.1)


a = ''
for year, studies in dict_timeline.items():
    if len(studies)>=2 and year != 2023 and year!= 2022:
        aa = ', '.join(studies[:len(studies)//2])
        ab = ', '.join(studies[len(studies)//2:])
        ac = aa + '\\\\ ' + ab 
    elif year == 2022:
        aa = ', '.join(studies[:len(studies)//3])
        ab = ', '.join(studies[len(studies)//3:2*len(studies)//3])
        abb = ', '.join(studies[2*len(studies)//3:])
        ac = aa + '\\\\ ' + ab + '\\\\ ' + abb
    else:
        ac = ', '.join(studies)

    if year == 2022:
        ad = '\\node[above right, align=left, rotate=45] at (11.15,'+delta+') {' + ac + '}; \n'
    else:
        ad = '\\node[above right, align=left, rotate=45] at (' + dict_year_point[year] + ','+delta+') {' + ac + '}; \n'
    a += ad

ba = ', '.join([f"{v}/{k}" for k, v in dict_year_point.items()])

ba + '\n'


b = '\\draw[dashed, thick] (-1.5,'+delta+') -- (0,'+delta+'); \n\
\\draw[->, thick] (0,'+delta+') -- (14,'+delta+'); \n\
\\foreach \\x/\\year in {' + ba + '} { \n\
    \\draw[shift={(\\x,'+delta+')}, color=black] (0pt,2pt) -- (0pt,-2pt); \n\
    \\node[below] at (\\x, '+delta+') {\\year}; \n\
    } \n'

c = '\\end{tikzpicture}'


d += b + a + c

print(d)

with open(os.path.join('tables', 'YN-timeline.tex'), 'w') as f:
    f.write(d)

\begin{tikzpicture} 
\draw[dashed, thick] (-1.5,0) -- (1,0); 
\draw[thick] (1,0) -- (13.5,0); 
\draw[dashed, thick] (13.5,0) -- (14,0); 
\foreach \x/\year in {-1/2002, 0.75/2008, 2.5/2009, 4.25/2010, 6.0/2011, 7.75/2012, 9.5/2013, 11.25/2014, 13.0/2015} { 
    \draw[shift={(\x,0)}, color=black] (0pt,2pt) -- (0pt,-2pt); 
    \node[below] at (\x, 0) {\year}; 
    } 
\node[above right, align=left, rotate=45] at (-1,0) {SoftWatt \cite{gurumurthi2002}}; 
\node[above right, align=left, rotate=45] at (0.75,0) {NU \cite{lewis2008}}; 
\node[above right, align=left, rotate=45] at (2.5,0) {NU \cite{singh2009} \cite{ma2009} \cite{spellmann2009}}; 
\node[above right, align=left, rotate=45] at (4.25,0) {LikwidPM \cite{treibig2010}\\ NU \cite{zamani2010}}; 
\node[above right, align=left, rotate=45] at (6.0,0) {NU \cite{suda2011} \cite{basmadjian2011} \cite{piga2011} \cite{matsumoto2011} \cite{chen2011}}; 
\node[above right, align=left, rotate=45] at (7.75,0) {ECAT \cite{chen2012}\\ eprof \cite{schube

## 4. Links list

In [17]:
links_list_latex = '\n '.join(df_YY.loc[df_YY['available_link'].str.contains('http'), 'for_links_list'].values) ## links from YY
links_list_latex += '\n '
links_list_latex += '\n '.join(df_YY.loc[df_YY['available_link_bis'].str.contains('http'), 'for_links_list_bis'].values) ## links from YY
links_list_latex += '\n '
links_list_latex += '\n '.join(df_YN.loc[df_YN['available_link'].str.contains('http'), 'for_links_list'].values) ## links from YN

# Final itemize for latex
links_list_latex = '\\begin{enumerate}[label={\\normalfont \\textbf{(L\\arabic*)}}] \n ' + links_list_latex + '\n\\end{enumerate}'
print(links_list_latex)

with open(os.path.join('tables', 'linklist.tex'), 'w') as f:
    f.write(links_list_latex)

\begin{enumerate}[label={\normalfont \textbf{(L\arabic*)}}] 
 \item \label{link-getzner2023} \url{https://github.com/JohannesGetzner/dl-energy-estimator}
 \item \label{link-you2023} \url{https://github.com/SymbioticLab/Zeus }
 \item \label{link-budennyy2022} \url{https://github.com/sb-ai-lab/Eco2AI }
 \item \label{link-carastan-santos2022} \url{https://github.com/phamthi1812/Benchmark-Tracker}
 \item \label{link-igescu2021} \url{https://github.com/epfl-iglobalhealth/CS433-2021-ecoML}
 \item \label{link-montgomerie-corcoran2021} \url{https://github.com/AlexMontgomerie/pommel}
 \item \label{link-lannelongue2021} \url{https://github.com/GreenAlgorithms/green-algorithms-tool}
 \item \label{link-wang2021} \url{https://github.com/Accelergy-Project/accelergy}
 \item \label{link-trebaol2020} \url{https://github.com/epfl-iglobalhealth/cumulator}
 \item \label{link-anthony2020} \url{https://github.com/lfwa/carbontracker }
 \item \label{link-henderson2020} \url{https://github.com/Breakend/experim

## 5. Subsidiary

In [18]:
_path = os.path.join('subsidiary.xlsx')
df_s = pd.DataFrame(pd.read_excel(_path))
df_s

df_1 = df_s[df_s['in_secondary']=='no']
df_1.loc[df_1['link'].notna(), 'code'] = ' \\href{' + df_1['link'].astype(str) + '}{\\ref*{link-c-' + df_1['name'].astype(str) + '}}'
df_1.loc[df_1['link_2'].notna(), 'doc'] = ' \\href{' + df_1['link_2'].astype(str) + '}{\\ref*{link-d-' + df_1['name'].astype(str) + '}}'
df_1.loc[df_1['link_3'].notna(), 'blog'] = ' \\href{' + df_1['link_3'].astype(str) + '}{\\ref*{link-b-' + df_1['name'].astype(str) + '}}'
df_1.loc[df_1['link'].notna(), 'for_link_list_1'] = '\\item \\label{link-c-' + df_1['name'].astype(str) + '} \\url{' + df_1['link'].astype(str) + '}'
df_1.loc[df_1['link_2'].notna(), 'for_link_list_2'] = '\\item \\label{link-d-' + df_1['name'].astype(str) + '} \\url{' + df_1['link_2'].astype(str) + '}'
df_1.loc[df_1['link_3'].notna(), 'for_link_list_3'] = '\\item \\label{link-b-' + df_1['name'].astype(str) + '} \\url{' + df_1['link_3'].astype(str) + '}'
# df_1.loc[df_1['full_name'].notna(), 'detail'] = ' (' + df_1['full_name'].astype(str) + ') ' + df_1['detail'].astype(str)
df_1.loc[df_1['full_name'].notna(), 'name'] =  df_1['name'].astype(str) + ' (' + df_1['full_name'].astype(str) + ')'


df_2 = df_s[(df_s['in_secondary']=='yes') & (df_s['tool_alone']=='yes')]
df_2.loc[df_2['link'].notna(), 'code'] = ' \\href{' + df_2['link'].astype(str) + '}{\\ref*{link-c-' + df_2['name'].astype(str) + '}}'
df_2.loc[df_2['link_2'].notna(), 'doc'] = ' \\href{' + df_2['link_2'].astype(str) + '}{\\ref*{link-d-' + df_2['name'].astype(str) + '}}'
df_2.loc[df_2['link'].notna(), 'for_link_list_1'] = '\\item \\label{link-c-' + df_2['name'].astype(str) + '} \\url{' + df_2['link'].astype(str) + '}'
df_2.loc[df_2['link_2'].notna(), 'for_link_list_2'] = '\\item \\label{link-d-' + df_2['name'].astype(str) + '} \\url{' + df_2['link_2'].astype(str) + '}'
df_2.loc[:, 'location'] = df_2['location'].apply(lambda x: '\cite{' + str(x) + '}').astype(str)
df_2.loc[df_2['full_name'].notna(), 'detail'] =  '(' + df_2['full_name'].astype(str) + ') ' + df_2['detail'].astype(str)

df_3 = df_s[(df_s['in_secondary']=='yes') & (df_s['tool_alone']=='no')]
df_3 = df_3.sort_values('year', ascending=False)
df_3['year'] = df_3['year'].astype(int)
df_3['general'] = df_3['fulltext_id'].apply(lambda x: '\cite{' + str(x) + '}').astype(str) + ', ' + df_3['year'].astype(str) + ', ' + df_3['name'].astype(str)
df_3['location'] = df_3['location'].apply(lambda x: '\cite{' + str(x) + '}').astype(str)
# df_3.loc[df_3['full_name'].notna(), 'detail'] = ' (' + df_3['full_name'].astype(str) + ') ' + df_3['detail'].astype(str)
df_3.loc[df_3['full_name'].notna(), 'name'] =  df_3['name'].astype(str) + ' (' + df_3['full_name'].astype(str) + ')'

def create_latex_1(df):

    links_list_latex = '\n '.join(df.loc[df['link'].notna(), 'for_link_list_1'].values)
    links_list_latex += '\n '
    links_list_latex += '\n '.join(df.loc[df['link_2'].notna(), 'for_link_list_2'].values)
    links_list_latex += '\n '
    links_list_latex += '\n '.join(df.loc[df['link_3'].notna(), 'for_link_list_3'].values)
    links_list_latex = '\\begin{enumerate}[label={\\normalfont \\textbf{(L\\arabic*)}}] \n\\setcounter{enumi}{25} \n ' + links_list_latex + '\n\\end{enumerate}'
    with open(os.path.join('tables', 'subsidiary_linklist_df1.tex'), 'w') as f:
        f.write(links_list_latex)

    list_cols = ['name', 'detail', 'code', 'doc', 'blog']
    a = '>{\\raggedright\\arraybackslash}'
    latex_col_width = '|'+a+'p{3.2cm}|p{6.5cm}|p{1.5cm}|p{1.5cm}|p{1.5cm}|'
    df = df[list_cols]
    print('nb: ', df.shape[0])
    df = df.fillna('')
    latex = df.to_latex(index=False).replace('llll', latex_col_width)
    with open(os.path.join('tables', 'subsidiary_1.tex'), 'w') as f:
        f.write(latex)

    return(latex + '\n' + links_list_latex)


def create_latex_2(df):

    links_list_latex = '\n '.join(df.loc[df['link'].notna(), 'for_link_list_1'].values)
    links_list_latex += '\n '
    links_list_latex += '\n '.join(df.loc[df['link_2'].notna(), 'for_link_list_2'].values)
    links_list_latex = '\\begin{enumerate}[label={\\normalfont \\textbf{(L\\arabic*)}}] \n\\setcounter{enumi}{36} \n ' + links_list_latex + '\n\\end{enumerate}'
    with open(os.path.join('tables', 'subsidiary_linklist_df2.tex'), 'w') as f:
        f.write(links_list_latex)

    list_cols = ['name', 'location', 'detail', 'code', 'doc']
    a = '>{\\raggedright\\arraybackslash}'
    latex_col_width = '|'+a+'p{3.25cm}|p{1.25cm}|p{6.7cm}|p{1.5cm}|p{1.5cm}|'
    df = df[list_cols]
    print('nb: ', df.shape[0])
    df = df.fillna('')
    latex = df.to_latex(index=False).replace('lllll', latex_col_width)
    with open(os.path.join('tables', 'subsidiary_2.tex'), 'w') as f:
        f.write(latex)

    return(latex + '\n' + links_list_latex)

def create_latex_3(df):
    list_cols = ['general', 'location', 'detail']
    a = '>{\\raggedright\\arraybackslash}'
    latex_col_width = '|'+a+'p{5cm}|p{1.6cm}|p{8.55cm}|'

    df = df[list_cols]

    print('nb: ', df.shape[0])

    df = df[list_cols]

    latex = df.to_latex(index=False).replace('lll', latex_col_width)

    with open(os.path.join('tables', 'subsidiary_3.tex'), 'w') as f:
        f.write(latex)
    return(latex)

l1 = create_latex_1(df_1)
l2 = create_latex_2(df_2)
l3 = create_latex_3(df_3)
print(l1)
print('\n')
print(l2)
print('\n')
print(l3)

nb:  5
nb:  7
nb:  26
\begin{tabular}{|>{\raggedright\arraybackslash}p{3.2cm}|p{6.5cm}|p{1.5cm}|p{1.5cm}|p{1.5cm}|l}
\toprule
name & detail & code & doc & blog \\
\midrule
Kepler (Kubernetes Efficient Power Level Exporter) & for Kubernetes systems, at process, container, or Kubernetes pod level, on Intel CPU, RAM and Nvidia GPU, or whole system, or whole network of systems &  \href{https://github.com/sustainable-computing-io/kepler}{\ref*{link-c-Kepler}} &  &  \href{https://sustainable-computing.io/}{\ref*{link-b-Kepler}} \\
Tracarbon & for Intel CPU and RAM &  \href{https://github.com/fvaleye/tracarbon}{\ref*{link-c-Tracarbon}} &  \href{https://fvaleye.github.io/tracarbon/documentation/}{\ref*{link-d-Tracarbon}} &  \href{https://medium.com/@florian.valeye/tracarbon-track-your-devices-carbon-footprint-fb051fcc9009}{\ref*{link-b-Tracarbon}} \\
PyJoules & for Intel CPU, RAM, and Nvidia GPU &  \href{https://github.com/powerapi-ng/pyJoules}{\ref*{link-c-PyJoules}} &  \href{https://pyjoules